In [1]:
import pandas as pd

from sklearn import svm,datasets
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

from Logistic_regression import X_test, y_train, y_test

iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
df['flower']=iris.target
df['flower']=df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [8]:
#the main or sole purpose of using hypertuning is to find a perfect high prediction score model to select based on judging them to do so:
#1> traditional way of using train_test_split
inputs=df.drop(['flower'],axis="columns")
target=df['flower']
X_train,X_test,y_train,y_test=train_test_split(inputs,target,test_size=0.2)
#then separately testing each model for its performance like:
#SVM
model = svm.SVC(kernel='rbf' ,C=30, gamma= 'auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8666666666666667

In [9]:
#since we do have multiple parameters in each model, to judge a model we also need to consider its parameters to do so we will be using k_fold crossval to get a perfect mean score of the model:-
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear',C=10, gamma='auto'), inputs, target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [10]:
cross_val_score(svm.SVC(kernel='rbf',C=10, gamma='auto'), inputs, target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [11]:
cross_val_score(svm.SVC(kernel='rbf',C=20, gamma='auto'), inputs, target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [14]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval, gamma='auto'),iris.data, iris. target, cv=5)
        avg_scores[kval + '_'+ str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': np.float64(0.9800000000000001),
 'rbf_10': np.float64(0.9800000000000001),
 'rbf_20': np.float64(0.9666666666666668),
 'linear_1': np.float64(0.9800000000000001),
 'linear_10': np.float64(0.9733333333333334),
 'linear_20': np.float64(0.9666666666666666)}

In [15]:
#from the above automation we can see different models scores with respective parameters
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear'],
},cv=5,return_train_score=False)
clf.fit(iris.data,iris.target)
clf.cv_results_


{'mean_fit_time': array([0.00095978, 0.00033383, 0.00026579, 0.00023561, 0.00026522,
        0.00025024]),
 'std_fit_time': array([1.01221865e-03, 5.10209966e-05, 1.03204130e-05, 1.63496051e-05,
        1.92624507e-05, 2.28387577e-05]),
 'mean_score_time': array([0.00032902, 0.00022011, 0.00017638, 0.00015483, 0.0001596 ,
        0.00015607]),
 'std_score_time': array([8.83581956e-05, 5.79731628e-05, 1.03237172e-05, 1.57980691e-05,
        1.55685689e-06, 8.78286930e-06]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
 

In [17]:
df=pd.DataFrame(clf.cv_results_)
df[[ 'param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [18]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [19]:
clf.best_score_

np.float64(0.9800000000000001)

In [22]:
#through this simple process using GridSearchCV we found the best combination
#HERE THE NEGATIVE POINT OF USING GRIDSEARCHCV IS THAT IT WILL ITERATE TO EACH VALUE PROVIDED IN THE C VALUE which will use high processing power in vain
#to solve this we use RandomSearchCV to achieve the best:
#which uses random values of the c in range and give a perfect combination
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm. SVC (gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf', 'linear']
},
cv=5, return_train_score=False, n_iter=2
)
rs.fit(iris.data,iris.target)
pd. DataFrame(rs.cv_results_) [['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,10,rbf,0.980000
1,10,linear,0.973333


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 5, 10]
        }
    }
}
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris

iris = load_iris()

scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

import pandas as pd
df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
print(df)


                 model  best_score                best_params
0                  svm    0.980000  {'C': 1, 'kernel': 'rbf'}
1        random_forest    0.960000       {'n_estimators': 10}
2  logistic_regression    0.966667                   {'C': 5}


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it t